# Get ready
Download nuget packages and extensions to investigate

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.21215.5"

# Lets load user feedback
Using `#!value` we laod the feedback and get the list of urls with probelms. Net we load is and explore as table with Nteract

In [1]:
#!value --name user_feedback --from-file ./report_export.json

In [1]:
#!share --from value user_feedback

In [1]:
using System.Text.Json;
var user_feedback_json = JsonDocument.Parse(user_feedback);

In [1]:
user_feedback_json.ExploreWithNteract().Display();

# Server logs
Using `#!value` we load the server logs and go to see what is going on using the explorers

In [1]:
#!value --name server_logs --from-file ./serverlog.json

In [1]:
#!share --from value server_logs

In [1]:
JsonDocument.Parse(server_logs).ExploreWithNteract().Display();

In [1]:
JsonDocument.Parse(server_logs).ExploreWithSandDance().Display();

# Data problem?
We can't spot anything here, so we need to see if there is anything that might cause rendering issues.
We use `#!connect` to connect to the database.

We try to get a picture by quering products and joining with some metadata like the categories.
Table view is not too informative, using EF context we look at that from SandDance

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

In [1]:
#!connect mssql --kernel-name adventureworks --create-dbcontext "Persist Security Info=False; Integrated Security=true; Initial Catalog=AdventureWorks2019; Server=localhost"

In [1]:
#!sql-adventureworks
SELECT 
product.Name,
product.ProductID,
category.ProductCategoryID,
product.ProductSubcategoryID,
category.Name
FROM AdventureWorks2019.Production.Product as product
INNER JOIN AdventureWorks2019.Production.ProductCategory as category on product.ProductSubcategoryID = category.ProductCategoryID

In [1]:
#!sql-adventureworks
select product.ProductID
,product.Name
,product.ProductNumber
,sub.ProductCategoryID
,sub.ProductSubcategoryID

from AdventureWorks2019.Production.Product as product
inner join AdventureWorks2019.Production.ProductSubcategory as sub
on product.ProductSubcategoryID = sub.ProductSubcategoryID

In [1]:
var reportedProducts = user_feedback_json.RootElement.EnumerateArray()
.Select(e => int.Parse( e.GetProperty("url").GetString().Split('/').Last() ))
.Distinct()
.ToHashSet();

In [1]:
var data =  from product in adventureworks.Product
            where reportedProducts.Contains( product.ProductId )
            join subCatebory in adventureworks.ProductSubcategory
            on product.ProductSubcategoryId equals subCatebory.ProductSubcategoryId
            select new { product.ProductId
                ,product.Name
                ,product.ProductNumber
                ,subCatebory.ProductSubcategoryId
                ,subCatebory.ProductCategoryId
                ,category = $"{subCatebory.ProductCategoryId}-{subCatebory.ProductSubcategoryId}"
            };

In [1]:
data.ExploreWithSandDance().Display();

# Nothing ...
So let's see if we get any problem related to comments sizes or count on those products

In [1]:
var reviewData =  from review in adventureworks.ProductReview
            join product in adventureworks.Product on review.ProductId equals product.ProductId
            join subCatebory in adventureworks.ProductSubcategory on product.ProductSubcategoryId equals subCatebory.ProductSubcategoryId
            where reportedProducts.Contains( product.ProductId )
            select new { product.ProductId
                ,product.Name
                ,product.ProductNumber
                ,subCatebory.ProductSubcategoryId
                ,subCatebory.ProductCategoryId
                ,category = $"{subCatebory.ProductCategoryId}-{subCatebory.ProductSubcategoryId}"
                ,CommentLength = review.Comments.Length
                ,review.Comments
            };

In [1]:
reviewData.ExploreWithSandDance().Display();

# Looks strange 
Why do we see a cluster of comments with exactly same lenght?

# Oh my ....
Let's create a filter on all data and let's see how much bigger this could actually be.

In [1]:
var problematicData = from review in adventureworks.ProductReview
join product in adventureworks.Product on review.ProductId equals product.ProductId
join subCatebory in adventureworks.ProductSubcategory on product.ProductSubcategoryId equals subCatebory.ProductSubcategoryId
where review.Comments.Contains("<script")
select new { product.ProductId
    ,product.Name
    ,product.ProductNumber
    ,subCatebory.ProductSubcategoryId
    ,subCatebory.ProductCategoryId
    ,category = $"{subCatebory.ProductCategoryId}-{subCatebory.ProductSubcategoryId}"
    ,review.Comments
    ,review.Rating
    ,review.EmailAddress
    ,review.ReviewerName
};

In [1]:
problematicData.Count()